In [ ]:
library(tidyverse) # For data manipulation and visualization
library(ggplot2)   # For visualizations
library(summarytools) # For detailed statistics
library(corrplot)  # For correlation plots
library(gridExtra) # For combining ggplots

In [4]:
#Load the data
churn_data = read.csv("Dataset/Customer Churn.csv")
head(churn_data)
summary(churn_data)
str(churn_data)


,Call..Failure,Complains,Subscription..Length,Charge..Amount,Seconds.of.Use,Frequency.of.use,Frequency.of.SMS,Distinct.Called.Numbers,Age.Group,Tariff.Plan,Status,Age,Customer.Value,Churn
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>
1,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,0
2,0,0,39,0,318,5,7,4,2,1,2,25,46.035,0
3,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,0
4,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,0
5,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0
6,11,0,38,1,3775,82,32,28,3,1,1,30,282.280,0


 Call..Failure      Complains       Subscription..Length Charge..Amount   
 Min.   : 0.000   Min.   :0.00000   Min.   : 3.00        Min.   : 0.0000  
 1st Qu.: 1.000   1st Qu.:0.00000   1st Qu.:30.00        1st Qu.: 0.0000  
 Median : 6.000   Median :0.00000   Median :35.00        Median : 0.0000  
 Mean   : 7.628   Mean   :0.07651   Mean   :32.54        Mean   : 0.9429  
 3rd Qu.:12.000   3rd Qu.:0.00000   3rd Qu.:38.00        3rd Qu.: 1.0000  
 Max.   :36.000   Max.   :1.00000   Max.   :47.00        Max.   :10.0000  
 Seconds.of.Use  Frequency.of.use Frequency.of.SMS Distinct.Called.Numbers
 Min.   :    0   Min.   :  0.00   Min.   :  0.00   Min.   : 0.00          
 1st Qu.: 1391   1st Qu.: 27.00   1st Qu.:  6.00   1st Qu.:10.00          
 Median : 2990   Median : 54.00   Median : 21.00   Median :21.00          
 Mean   : 4472   Mean   : 69.46   Mean   : 73.17   Mean   :23.51          
 3rd Qu.: 6478   3rd Qu.: 95.00   3rd Qu.: 87.00   3rd Qu.:34.00          
 Max.   :17090   Max.   :

'data.frame':	3150 obs. of  14 variables:
 $ Call..Failure          : int  8 0 10 10 3 11 4 13 7 7 ...
 $ Complains              : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Subscription..Length   : int  38 39 37 38 38 38 38 37 38 38 ...
 $ Charge..Amount         : int  0 0 0 0 0 1 0 2 0 1 ...
 $ Seconds.of.Use         : int  4370 318 2453 4198 2393 3775 2360 9115 13773 4515 ...
 $ Frequency.of.use       : int  71 5 60 66 58 82 39 121 169 83 ...
 $ Frequency.of.SMS       : int  5 7 359 1 2 32 285 144 0 2 ...
 $ Distinct.Called.Numbers: int  17 4 24 35 33 28 18 43 44 25 ...
 $ Age.Group              : int  3 2 3 1 1 3 3 3 3 3 ...
 $ Tariff.Plan            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Status                 : int  1 2 1 1 1 1 1 1 1 1 ...
 $ Age                    : int  30 25 30 15 15 30 30 30 30 30 ...
 $ Customer.Value         : num  198 46 1537 240 146 ...
 $ Churn                  : int  0 0 0 0 0 0 0 0 0 0 ...
